In [1]:
!pip install gluonnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595737 sha256=562f368490fec6ee340f218d5dfb145fbcf9617435a44b26609ef3816c45b96d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


In [2]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 153 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-p6uggj5a
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-p6uggj5a
     |████████████████████████████████| 129 kB 5.1 MB/s 
     |████████████████████████████████| 54.7 MB 18 kB/s 
     |████████████████████████████████| 4.5 MB 62.6 MB/s 
     |████████████████████████████████| 1.2 MB 51.1 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x3a1fe000 @  0x7f8467f5e615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 1

In [4]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch import optim
import numpy as np

In [5]:
import gluonnlp as nlp

In [6]:
from tqdm import tqdm, tqdm_notebook

In [7]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
from kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# GPU 설정
device = torch.device("cuda:0")

In [11]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [12]:
#Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=20,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
model = torch.load('/content/drive/MyDrive/last_final_model.pt')
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [17]:
def predict(predict_sentence,model):
    
    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("편안함")
            elif np.argmax(logits) == 1:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 2:
                test_eval.append("행복")
            elif np.argmax(logits) == 3:
                test_eval.append("감사")
            elif np.argmax(logits) == 4:
                test_eval.append("만족")
            elif np.argmax(logits) == 5:
                test_eval.append("흥미로움")
            elif np.argmax(logits) == 6:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 7:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 8:
                test_eval.append("당황")
            elif np.argmax(logits) == 9:
                test_eval.append("상처")
            elif np.argmax(logits) == 10:
                test_eval.append("불안")
            elif np.argmax(logits) == 11:
                test_eval.append("짜증")
            elif np.argmax(logits) == 12:
                test_eval.append("답답")
            elif np.argmax(logits) == 13:
                test_eval.append("허무함")
            elif np.argmax(logits) == 14:
                test_eval.append("조롱")
            elif np.argmax(logits) == 15:
                test_eval.append("불편함")
            elif np.argmax(logits) == 16:
                test_eval.append("걱정")
            elif np.argmax(logits) == 17:
                test_eval.append("불만")
            elif np.argmax(logits) == 18:
                test_eval.append("무서움")
            elif np.argmax(logits) == 19:
                test_eval.append("보통")
            else :
                test_eval.append("궁금")

        
    return test_eval[0]

#        print('오늘의 기분은'+test_eval[0])

In [18]:
# text = open('/content/drive/MyDrive/text_file/test.txt', 'r')
# text = text.read().splitlines()
# print(text)

In [19]:
# str = ''.join(text)

In [20]:
# a = predict(str, model)

In [21]:
heal = ['여수 밤바다', '파주 평화누리공원', '순천 갈대밭', '제주도 해안도로', 
         '진주 진양호', '장흥유원지', '대구 앞산정만대', '춘천 해피초원농장' ,' 속초 해수욕장', 
         '경주 불국사', '포항 호미곶', '남해 두모마을', '대관령 하늘목장', '군산 철길마을', 
         '국립 광릉수목원', '금선사 템플스테이', '보성 제암산자연휴양림','군산 선유도', '용인 에버랜드',
        '잠실 롯데월드', '경주 경주월드', '대구 이월드', '부산 롯데월드', '남원 광한루', '부산 해운대',
        '광안리해수욕장', '다대포해수욕장', '송정해수욕장', '거제도 매미성', '통영 해파랑길',
        '곡성 기차마을', '임실 치즈마을', '대관령 양떼목장', '남원 서도역', '정동진해수욕장',
        '강릉 강문해변', '담양 죽녹원', '전주 한옥마을', '제주 올레길', '순천 순천만', '춘천 남이섬', '안동 하회마을',
        '수원화성', '부산 감천문화마을', '독도', '창녕 우포늪', '합천 해인사', '서울 5대궁궐',
        '남산타워', '경주 황리단길', '신사동 가로수길', '이태원 경리단길']
 
extreme= ['통영 어드벤처 타워', '제주도  스쿠버다이빙', '단양 패러글라이딩', 
          '강원 내린천 래프팅', '충주 스카이다이빙', '하남 스포츠몬스터', '서울 한강 워터 제트팩', 
          '일산 인공 서핑', '영월 동강래프팅', '여수 스카이플라이', '문경 패러글라이딩', 
          '경남 하동 하동알프스레포츠', '인천 스카이 짚라인', '강화 루지', '통영 루지', '해운대 스카이캡슐',
          '광안리 요트투어', '광안리 해양레포츠', '송도 케이블카', '여수 케이블카']
 
movie=['크루엘라', '매드맥스', '탑건', '하울의 움직이는 성', '센과 치히로의 행방불명', '토이스토리', '이웃집토토로', 
       '업', '모아나', '겨울왕국', '코코', '인사이드 아웃', '너의 이름은', '범죄도시', '극한직업', '신과함께', '어벤져스',
       '미션임파서블', '스파이더맨', '베놈', '알라딘', '주토피아', '데드풀', '마루 밑 아리에티', '몬스터주식회사', '곰돌이 푸',
       '니모를 찾아서', '7번방의 선물', '늑대소년', '기생충', '미나리', '괴물', '해운대', '왕의남자', '변호인', '형', '엑시트',
       '명량', '한산', '건축학개론', '도둑들', '베테랑', '암살', '부산행', '관상', '택시운전사', '태극기 휘날리며', '청년경찰',
       '스물', '더 테러 라이브', '검은사제들', '검사외전', '전우치', '추격자']
 

k_balad=['Timeless - SG워너비',
'밤하늘의 별을 (2020) - 경서',
'어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 - AKMU (악동뮤지션)',
'서울의 잠 못 이루는 밤 (Feat. 이수현) - 10CM',
'잠이 오질 않네요 - 장범준',
'I Love U-성시경',
'내사람 (Partner For Life) - SG워너비',
'취기를 빌려 - 산들',
'안녕 (Hello) - 조이',
'밤편지 - 아이유(IU)',
'너의 모든 순간 - 성시경',
'좋을텐데 (If Only) (Feat. 폴킴) - 조이',
'봄 안녕 봄 - 아이유(IU)',
'Love poem - 아이유(IU)',
'아이와 나의 바다 - 아이유(IU)',
'벌써 일년 - 반하나& MJ(써니사이드)',
'이렇게 좋아해 본 적이 없어요 - CHEEZE (치즈)',
'헤픈우연 - 헤이즈',
'어린왕자 - 려욱',
'광화문에서 - 규현',
'나비무덤 - Take',
'청소 - The Ray',
'나를 잊지 말아요 - 허각',
'사월의 눈 - 허각',
'보고싶은날엔 - 박지헌',
'그녀를 사랑해줘요 - 하동균',
'저녁하늘 - 에일리',
'가을안부 - 먼데이키즈',
'시간과 낙엽 - 악동뮤지션',
'새벽가로수길(feat.송유빈) - 백지영',
'너였다면 - 정승환',
'선물 - 멜로망스',
'이럴거면 그러지말지 - 백아연',
'안아줘 - 정준일',
'거북이 - 다비치',
'모든날 모든순간 - 폴킴',
'취중고백 - 멜로망스',
'오래된 노래 - 스텐딩에그',
'Little star - 스텐딩에그',
'All of my life - 박원',
'두사랑 - 다비치',
'아로하 - 조정석',
'미워도 사랑하니까 - 다비치',
'기대 - 나윤권',
'그라데이션 - 10cm',
'그리워하다 - 비투비',
'봄날의 기억 - 비투비',
'널 사랑하지않아 - 어반자카파'
'눈의 꽃 - 박효신',
'기억을 걷는 시간 - 넬'
]
 
k_dance=[
'Butter - 방탄소년단',
'Next Level - aespa',
'Dun Dun Dance - 오마이걸(OH MY GIRL)',
'치맛바람 (Chi Mat Ba Ram) - 브레이브걸스(Brave Girls)',
'Alcohol-Free - TWICE(트와이스)',
"롤린 (Rollin') - 브레이브걸스(Brave Girls)",
'라일락 - 아이유(IU)',
'ASAP - STAYC(스테이씨)',
'Dynamite - 방탄소년단',
'상상더하기 - MSG워너비',
'Celebrity - 아이유(IU)',
'상상더하기 - 라붐(LABOUM)',
'Ready to love - 세븐틴',
'Dolphin - 오마이걸(OH MY GIRL)',
'Lovesick Girls - BLACKPINK',
'어푸(Ah puh) - 아이유(IU)',
'Loving U - 씨스타',
'Hot issue - 포미닛',
'왜(Keep your head down) - 동방신기',
'Catch Me - 동방신기',
'Shock - 비스트',
'남자가 사랑할 때 - 인피니트',
'추격자 - 인피니트',
'내꺼하자 - 인피니트',
'fiction - 비스트',
'파라다이스 - 인피니트',
'Bad - 인피니트',
'으르렁 - 엑소',
'MAMA - 엑소',
'중독 - 엑소',
'다칠 준비가 돼있어 - 빅스',
'기적 - 빅스',
'도원경 - 빅스',
'Hide - 빅스',
'movie - 비투비',
'After Like - 아이브',
'Eleven - 아이브',
'Love Dive - 아이브',
'Attention - 뉴진스',
'카니발 - 가인',
'피어나 - 가인',
'식스센스 - 브라운아이드걸스',
'종소리 - 러블리즈',
'다시만난세계 - 소녀시대',
'도깨비불 - aespa',
'Hype boy - 뉴진스',
'Goodbye Summer - 에프엑스',
'첫사랑니 - 에프엑스',
'4walls - 에프엑스',
'Hello future - 엔시티드림',
'Happiness - 레드벨벳',
'러시안룰렛 - 레드벨벳',
'View - 샤이니',
'Sherlock - 샤이니',
'Heartbeat - 2PM',
'우리집 - 2PM',
'Tell me - 원더걸스',



]
 
k_hip=[
'마.피.아. In the morning-ITZY(있지)',
'맛 (Hot Sauce)-NCT DREAM',
'밸런스 게임-투모로우바이투게더',
'GAM3 BO1-세븐틴',
'비도 오고 그래서 (Feat. 신용재)-헤이즈 (Heize)',
'METEOR-창모(CHANGMO)',
'DNA-방탄소년단',
'IDOL-방탄소년단',
'FAKE LOVE-방탄소년단',
'피 땀 눈물-방탄소년단',
'사이렌-호미들',
'멜로디-ASH ISLAND',
'I NEED U-방탄소년단',
'아무노래-지코 (ZICO)',
'어떻게 지내 (Prod. By VAN.C)-오반(OVAN)',
'Rainy day (Feat. ASH ISLAND, Skinny Brown)-PATEKO(파테코)',
'뚜두뚜두 (DDU-DU DDU-DU)-BLACKPINK',
'죽일놈 - 다이나믹듀오',
'불꽃놀이 - 다이나믹듀오',
'TV를 껐네(feat. Tasha, Kwon Jungyeol OF 10CM) - 리쌍',
'Forever - 비아이',
'아무노래 - 지코',
'너는 나 나는 너 - 지코',
'거북선(feat.팔로알토) - Ja Mezz, Andup (앤덥), MINO (송민호)',
'Born Hater - 에픽하이',
'Fly - 에픽하이',
'Love Love Love - 에픽하이',
'그땐 그땐 그땐 - 슈프림팀',
'왜 - 슈프림팀',
'노땡큐 (Feat. MINO, 사이먼 도미닉, 더콰이엇) - 에픽하이',
'party(SHUT DOWN)(feat. 크러쉬(Crush)) - 식케이',
'flex - 기리보이',
'시차 - 우원재',
'바코드 - 김하온&이병재',
'리무진 (Feat. MINO) - 비오',
'아쿠아맨 - 빈지노',
'회전목마 (Feat. Zion.T, 원슈타인) - 소코도모 ',
'보트 (Boat) - 죠지',
'외톨이 - 아웃사이더',
'몽환의숲(feat.이루마) - 키네틱플로우',
'인스타그램 - 딘',
'D(half moon) - 딘',
'Red sun - 행주',
'Jackpot - 블락비'
]


food = ["삼계탕", "삼겹살", "곱창", "찜닭", "오리고기", "소고기", 
          "국밥", "닭도리탕", "낙곱새", "라면", "비빔밥", "칼국수", 
          "수제비", "갈비", "제육볶음", "스테이크", "파스타", "필라프", "감바스", "리조또", "샐러드", 
           "피자", "빠에야", "플래터", "스튜", "짜장면", "뿌팟퐁커리", "팟타이", "나시고랭", "쌀국수", "미고랭",
         "카레", "마라탕", "마라샹궈", "훠궈", "돈까스", "월남쌈", "라멘", 
         "탄탄멘", "규동", "꿔바로우", "똠양꿍", "물냉면"]

spicy = ["엽떡 3단계", "불닭볶음면", "송주불냉면", "실비김치", "불닭발", 
         "김치찜", "김치찌개", "감자탕", "짬뽕", "닭발", "부대찌개",
         "순두부찌개", "아구찜", "해물찜", "육개장", "낙지볶음", "쭈꾸미", 
         "돼지갈비찜", "소꼬리찜", "비빔냉면", "짚신매운갈비찜"]

dessert = ["와플", "마카롱", "빙수", "크로크모슈", "케이크", "허니바게트볼",
           "치즈케이크", "레몬파운드케이크", "쿠키", "허니브레드", "오믈렛", "베이글",
           "휘낭시에", "초코케이크", "말차스콘", "마들렌", "츄러스", "다쿠아즈", "초코브라우니",
           "도넛", "꽈배기", "카스테라", "소금빵", "크로플", "크로넛", "에그타르트", "크로넛", "크로와상"]

snack = ["닭강정", "양꼬치", "핫윙", "소떡소떡", "가라아게", "콘치즈", "감자튀김", 
         "치킨너겟", "치킨", "나쵸", "소시지", "버터오징어구이", "핫도그", 
         "해쉬브라운", "맥모닝", "맥치킨", "맥윙", "스낵랩", "치즈스틱", "꼬깔콘", '프링글스',
         "포카칩", '빼빼로', '타코야끼', '오코노미야끼', '김치전', '감자전', '부추전', '뻥튀기',
         '약과', '군밤', '붕어빵', '씨앗호떡', '호두과자', '계란빵', '핫바', '젤리', '아이셔',
         '새콤달콤', '치즈볼', '쥐포', '콜팝', '빈대떡', '순대']

coffee = ["아메리카노", "콜드브루", "바닐라라떼", "카페라떼", "카라멜마키아또",
          "카페모카", "카페모카프라페", "연유라떼", "화이트 모카",
          "민트모카", "헤이즐넛라떼", "에스프레소", '아인슈패너', '플랫화이트', '맥심아이스커피',
          '달고나라떼', '아샷추', '레쓰비']

beverage = ["초코 라떼", "민트초코 라떼", "밀크티", "흑당 버블티", "레몬차", "자몽차", '허니자몽블랙티',
            "유자차", "모히토", "요거트 스무디", "블루베리 스무디", "딸기 스무디", "애플망고 스무디",
            "레몬 에이드", "자몽 에이드", "딸기 생과일 주스", "오곡 프라페", "쿠앤크 프라페",
            '유자애플티', '타로버블티', '오레오쿠키스무디', '밀크쉐이크', '생초콜릿라떼', '딸기쉐이크',
            '청포도에이드', '딸기바나나주스', "키위 생과일 주스", '사과케일주스', '비타500', '알로에주스',
            '식혜', '컨피던스', '소주', '맥주', '칵테일', '와인', '막걸리', '샴페인']



f0 = [heal,extreme, movie,k_dance,k_hip,spicy,dessert,snack,coffee,beverage, food]
f1 = [extreme,movie,k_dance,k_hip,spicy,dessert,snack,coffee,beverage]
f2 = [heal,movie,k_dance,k_balad,dessert,food,beverage]

In [22]:
# import sys
 
# def printsave(text):
#     file = open('/content/drive/MyDrive/result/result.txt','w')
#     print(text)
#     print(text,file=file)
#     file.close()

In [23]:
# import random

# if (a == '편안함') | (a == '기쁨') | (a == '행복') | (a == '감사') | (a == '만족') | (a == '흥미로움') | (a == '보통'):
#   pick = random.choice(f0)
#   print('제가 생각하기에는 오늘의 감정은 '+ a + '인 것 같아요!\n오늘 {0}을(를) 추천드려요!'.format(random.choice(pick)))

# elif (a == '분노') | (a == '불만') | (a == '짜증') | (a == '답답')| (a == '조롱'):
#   pick = random.choice(f1)
#   print('제가 생각하기에는 오늘의 감정은 '+ a + '인 것 같아요.\n 오늘 {0} 어때요?'.format(random.choice(pick)))

# else:
#   pick = random.choice(f2)
#   printsave('제가 생각하기에 오늘의 감정은 '+ a + '인 것 같아요.\n오늘의 그대를 위해 {0} 어때요?\n오늘도 수고했어요 언제나 그대를 응원합니다.'.format(random.choice(pick)))

In [24]:
# 최종코드 
import threading
import os
import sys
def printsave(text):
    file = open('/content/drive/MyDrive/result/result.txt','w')
    print(text)
    print(text,file=file)
    file.close()

def emotion():
  # timer = threading.Timer(5, emotion)
  # timer.start()
  while True:
    if os.path.isfile("/content/drive/MyDrive/text_file/test.txt"):
      text = open('/content/drive/MyDrive/text_file/test.txt', 'r')
      text = text.read().splitlines()
      # print(text)

      str = ''.join(text)
      a = predict(str, model)

      import random

      if (a == '편안함') |  (a == '감사') | (a == '만족') | (a == '흥미로움') | (a == '보통'):
        pick = random.choice(f0)
        printsave('제가 생각하기에 오늘의 감정은 '+ a + '인 것 같아요!\n오늘 {0}을(를) 추천드려요!\n그대의 내일은 더욱더 빛나길 기도할게요.❤'.format(random.choice(pick)))


      elif (a == '기쁨') | (a == '행복') :
        pick = random.choice(f0)
        printsave('제가 생각하기에 오늘의 감정은 '+ a + '인 것 같아요!\n너무 부럽네요!\n오늘 {0} 고?😎 '.format(random.choice(pick)))

      elif (a == '분노') | (a == '불만') | (a == '짜증') | (a == '답답'):
        pick = random.choice(f1)
        printsave('제가 생각하기에 오늘의 감정은 '+ a + '인 것 같아요.\n오늘 {0} 어때요?\n기분이 한결 나아질 거에요😀!'.format(random.choice(pick)))

      elif  (a=='불안'):
          printsave('생각이 많을땐 레몬사탕이지!\n레몬사탕 먹고 털어버려요!🍋')

      elif (a == '조롱'):
          pick = random.choice(f1)
          printsave('누군가를' +a+ '하는 당신, 당신이 있어 오늘 하루도 따뜻합니다.😁 \n 그런 의미에서 오늘 {0} 로 땡길까요?')

      else:
        pick = random.choice(f2)
        printsave('제가 생각하기에 오늘의 감정은 '+ a + '인 것 같아요.\n오늘의 그대를 위해 {0} 어때요?\n오늘도 수고했어요 언제나 그대를 응원할게요.🎈'.format(random.choice(pick)))

      break
    else : 
      continue

In [28]:
emotion()
os.remove('/content/drive/MyDrive/text_file/test.txt')

제가 생각하기에 오늘의 감정은 행복인 것 같아요!
너무 부럽네요!
오늘 시차 - 우원재 고?😎 
